# TheGraph data access
We plan to visualize and monitor the flow of delegated stake, total delegation pool and indexer allocation process. 

The current data come from [event data subgraph](https://thegraph.com/explorer/subgraph/fordn/graph-network-events?selected=playground)

More research plan see [hackmd](https://hackmd.io/@Jiajia20/BkOoEZODd)


## visualization
- request GraphQL data API -> JSON -> PANDAS
- the current subgraph seemd only load the first 200 record (even if I specify first:1000)
- I'm able to load 200 at a time and combine them with 'first, skip' 

## To do
- [x] get specitialty subgraph
- [x] double check pagination problem
- [x] clean the notebook and upload


## 0 -  Pagination

In [1]:
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt

In [24]:
def query_theGraph(raw_query, field_name, url, verbose=False):

    query_parts =raw_query.split(')')
    paginator = ", where: {}"
    #this expectes the raw query to gave a `first:1000` term
    n = 0
    records = []
    while n < 1:
        if (n == 0):
            print(query_parts[0])
            print(query_parts[1])
            
            query = 'query '+query_parts[0] + 'id:' + str(n) + ')' + query_parts[1]
            'orderBy:blockNumber, where:{blockNumber_gt: 0})' 
            print(query)
        else:
            #print(f'request {n+1}')
            query = 'query '+query_parts[0] + 'id:' + str(n) + ')' + query_parts[1]

        if verbose:
            print(query)

        r = requests.post(url, json = {'query':query})
        print(r)
        try:
            d = json.loads(r.content)['data'][field_name]
        except:
            print(r.content)
            errors = json.loads(r.content)['errors']
            print(errors)
            for e in errors:
                print(e['message'])

        #print(f'results {len(d)}')
        records.extend(d)
        #print(f'total {len(records)}')
        try:
            indexer = d[-1]['blockNumber']
        except:
            print('noblock')
        
        n += 1
        if len(d) < 99:
            break
        
    return pd.DataFrame(records)

## 1 - Allocation Data of Indexer


In [25]:
url = 'https://api.thegraph.com/subgraphs/name/fordn/graph-network-events'

created_query = '''query{
  epoch() {
    id
    startBlock
    endBLock
  }  
}
'''
#r = requests.post(url, json = {'query':raw_query})
#d = json.loads(r.content)['data'][field_name]
#pd.DataFrame(d)
field_name = 'epoch'

df_epoch = query_theGraph(created_query, field_name, url, True)
df_epoch

query{
  epoch(
 {
    id
    startBlock
    endBLock
  }  
}

query query{
  epoch(id:0) {
    id
    startBlock
    endBLock
  }  
}

query query{
  epoch(id:0) {
    id
    startBlock
    endBLock
  }  
}

<Response [200]>
b'{"errors":[{"locations":[{"line":2,"column":3}],"message":"Type `Query` has no field `epoch`"}]}'
[{'locations': [{'line': 2, 'column': 3}], 'message': 'Type `Query` has no field `epoch`'}]
Type `Query` has no field `epoch`


UnboundLocalError: local variable 'd' referenced before assignment

In [17]:
df_epoch['type'] = 'epoch'
df_epoch

NameError: name 'df_epoch' is not defined

In [5]:
df_epoch.to_csv('epoch.csv', index = False, header = True)